In [19]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [21]:
tf.keras.backend.clear_session()

In [22]:
def add_cos_monthly(df, column_name):
    values = [np.cos(2 * np.pi * (i % 12) / 12) for i in range(len(df))]
    df[column_name] = values
    return df

In [23]:
subject = 'Goiás - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [24]:
data = pd.read_csv('2003_mo_model_input_GO.csv')
data = add_cos_monthly(data, "Sazonalidade")
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Sazonalidade,Goiás - Consumo de Cimento (t)
0,2003-1,56.459960,0.745293,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,0.260097,9.371822e+07,5.518325e+06,14.031270,8.418248e+07,8.179785,1.000000e+00,97.447
1,2003-2,56.789152,0.745405,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,0.258964,9.383590e+07,5.523290e+06,14.034476,8.422082e+07,8.174086,8.660254e-01,97.256
2,2003-3,57.140263,0.745517,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,0.257352,9.395358e+07,5.528254e+06,14.037682,8.425916e+07,8.168386,5.000000e-01,101.442
3,2003-4,57.649192,0.745629,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,0.255929,9.407126e+07,5.533219e+06,14.040888,8.429750e+07,8.162687,6.123234e-17,98.217
4,2003-5,58.213295,0.745741,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,0.254109,9.418894e+07,5.538184e+06,14.044093,8.433584e+07,8.156988,-5.000000e-01,112.540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,175.674487,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.520178,NaN,NaN,NaN,NaN,NaN,-8.660254e-01,274.673
236,2022-9,174.614843,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.518515,NaN,NaN,NaN,NaN,NaN,-5.000000e-01,247.240
237,2022-10,173.307223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.516447,NaN,NaN,NaN,NaN,NaN,-1.836970e-16,246.647
238,2022-11,171.556538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.513912,NaN,NaN,NaN,NaN,NaN,5.000000e-01,225.813


In [25]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Sazonalidade
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818,1.414214e+00
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374,1.224745e+00
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930,7.071068e-01
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487,1.514512e-16
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043,-7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503,-1.224745e+00
188,1.677410,0.682578,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711,-7.071068e-01
189,1.732332,0.638983,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918,-1.949312e-16
190,1.763870,0.595389,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126,7.071068e-01


In [26]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Goiás - Consumo de Cimento (t), Length: 240, dtype: float64

In [27]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Sazonalidade
0,-1.656649,-2.298583,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.950106,-1.718818,-1.724379,-2.510500,-2.091868,-0.400818,1.414214e+00
1,-1.644539,-2.259040,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.970613,-1.700304,-1.668988,-2.459954,-2.059753,-0.411374,1.224745e+00
2,-1.631624,-2.219497,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.999751,-1.681790,-1.613597,-2.409407,-2.027637,-0.421930,7.071068e-01
3,-1.612902,-2.179955,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.025491,-1.663277,-1.558206,-2.358860,-1.995522,-0.432487,1.514512e-16
4,-1.592151,-2.140412,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.058403,-1.644763,-1.502815,-2.308314,-1.963406,-0.443043,-7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.623245,1.561651,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,0.944351,1.175698,-1.121085,-0.108013,0.955652,1.757990,1.224745e+00
158,0.644558,1.540971,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,0.952621,1.178649,-1.137848,-0.116498,0.956673,1.696610,7.071068e-01
159,0.663382,1.520292,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,0.960982,1.181601,-1.154611,-0.124984,0.957693,1.635230,1.514512e-16
160,0.670170,1.499612,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,0.969363,1.184552,-1.171374,-0.133469,0.958714,1.573850,-7.071068e-01


In [28]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       93.660
1       85.349
2      116.097
3      120.197
4      122.728
        ...   
157    146.310
158    191.374
159    164.318
160    211.049
161    198.789
Name: Goiás - Consumo de Cimento (t), Length: 162, dtype: float64

In [29]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [30]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 16)

In [31]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,Goiás - Produção de Cimento (t),Goiás - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Goiais-03anhão - value,Goiás - PIB - Estadual,Goiás - PIB - Construção Civil,Goiás - PIB - Per Capita,Goiás - PIB - Preços de Mercado,Goiás - Desemprego,Sazonalidade
126,0.798828,0.605338,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.673675,0.830615,0.695340,0.936426,0.971320,0.568035,-1.414214e+00
127,0.774031,0.608897,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.687412,0.848214,0.652086,0.905979,0.976868,0.664626,-1.224745e+00
128,0.731137,0.612457,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,0.706891,0.865813,0.608833,0.875531,0.982416,0.761218,-7.071068e-01
129,0.695886,0.616016,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,0.722784,0.883412,0.565579,0.845084,0.987963,0.857809,-1.949312e-16
130,0.655420,0.619576,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,0.738712,0.901011,0.522326,0.814636,0.993511,0.954400,7.071068e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,1.639330,0.726172,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.221276,1.118822,-1.173185,-0.446937,0.837128,0.999503,-1.224745e+00
188,1.677410,0.682578,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.236987,1.111624,-1.156891,-0.447051,0.832077,1.003711,-7.071068e-01
189,1.732332,0.638983,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.252847,1.104427,-1.140596,-0.447166,0.827027,1.007918,-1.949312e-16
190,1.763870,0.595389,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.276241,1.097229,-1.124301,-0.447280,0.821977,1.012126,7.071068e-01


In [32]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 16)

In [33]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [34]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [35]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [36]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[1763006249, 815377871, 3727194152, 1827892079, 288343232, 2898296216, 3863459888, 2801935343, 2199365661, 1328558864, 2138621935, 1380998812, 3038559550, 3918601041, 1071149039, 1699132260, 661093650, 2151326974, 1919074500, 1113361999, 1058318561, 302224744, 2245775064, 3654871603, 4193257323, 2562773454, 3016086774, 1391372534, 18672274, 2549329489, 1227387524, 1272668676, 1956858422, 3029167791, 3627802166, 2737779358, 1732449773, 1133661878, 3865616376, 1034899949, 2591267193, 3754393158, 29526959, 3817723837, 3354507308, 47285477, 2035502435, 2637516822, 868851874, 3442868896]


Step: 0 ___________________________________________
val_loss: 455.5384216308594
winner_seed: 1763006249


Step: 1 ___________________________________________
val_loss: 420.35198974609375
winner_seed: 815377871


Step: 2 ___________________________________________
val_loss: 408.2400817871094
winner_seed: 3727194152


Step: 3 ___________________________________________
val_loss: 406.6273498535156
winner_see

In [37]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 4s 204ms/step - loss: 40395.9570 - val_loss: 18932.7109
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 6460.9634 - val_loss: 1091.8351
Epoch 3/10000
4/4 [==============================] - 0s 10ms/step - loss: 1563.8175 - val_loss: 1415.0953
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 1380.0673 - val_loss: 1115.5704
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 1185.2931 - val_loss: 1221.7200
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 1367.0587 - val_loss: 1200.4814
Epoch 7/10000
4/4 [==============================] - 0s 9ms/step - loss: 1559.0576 - val_loss: 1195.2878
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 1241.2407 - val_loss: 1121.7659
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 1139.1404 - val_loss: 1023.1501
Epoch 10/10000
4/4 [==========================

4/4 [==============================] - 0s 9ms/step - loss: 235.4856 - val_loss: 639.7642
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 234.4832 - val_loss: 490.4207
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 160.2426 - val_loss: 462.7545
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 142.9758 - val_loss: 439.7746
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 135.7090 - val_loss: 457.5587
Epoch 84/10000
4/4 [==============================] - 0s 9ms/step - loss: 140.9326 - val_loss: 429.5208
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 135.9071 - val_loss: 440.7013
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 146.8055 - val_loss: 492.3509
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 144.2800 - val_loss: 455.6335
Epoch 88/10000
4/4 [==============================] - 0s 9ms/step - loss: 142.6

4/4 [==============================] - 0s 9ms/step - loss: 113.3904 - val_loss: 453.3443
Epoch 158/10000
4/4 [==============================] - 0s 9ms/step - loss: 99.3968 - val_loss: 382.7863
Epoch 159/10000
4/4 [==============================] - 0s 9ms/step - loss: 89.0753 - val_loss: 388.8713
Epoch 160/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.8576 - val_loss: 371.4934
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.4284 - val_loss: 406.7258
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 82.8793 - val_loss: 443.6481
Epoch 163/10000
4/4 [==============================] - 0s 9ms/step - loss: 80.7532 - val_loss: 525.5172
Epoch 164/10000
4/4 [==============================] - 0s 9ms/step - loss: 141.6275 - val_loss: 385.1767
Epoch 165/10000
4/4 [==============================] - 0s 9ms/step - loss: 101.5750 - val_loss: 378.5627
Epoch 166/10000
4/4 [==============================] - 0s 9ms/step - loss: 10

Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 62.5233 - val_loss: 368.9535
Epoch 237/10000
4/4 [==============================] - 0s 9ms/step - loss: 71.2513 - val_loss: 397.5316
Epoch 238/10000
4/4 [==============================] - 0s 9ms/step - loss: 64.8156 - val_loss: 422.8245
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 70.0809 - val_loss: 420.5055
Epoch 240/10000
4/4 [==============================] - 0s 9ms/step - loss: 63.1954 - val_loss: 424.2287
Epoch 241/10000
4/4 [==============================] - 0s 9ms/step - loss: 62.0063 - val_loss: 414.7609
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step - loss: 61.2177 - val_loss: 408.7287
Epoch 243/10000
4/4 [==============================] - 0s 9ms/step - loss: 68.3545 - val_loss: 418.9913
Epoch 244/10000
4/4 [==============================] - 0s 9ms/step - loss: 55.2260 - val_loss: 401.1198
Epoch 245/10000
4/4 [==============================] - 0s 9ms/st

4/4 [==============================] - 0s 9ms/step - loss: 52.9870 - val_loss: 403.5481
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.3628 - val_loss: 402.8133
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 45.0893 - val_loss: 380.5653
Epoch 317/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.6266 - val_loss: 411.6120
Epoch 318/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.8366 - val_loss: 400.7735
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 52.1178 - val_loss: 392.7780
Epoch 320/10000
4/4 [==============================] - 0s 9ms/step - loss: 63.0062 - val_loss: 438.1979
Epoch 321/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.2773 - val_loss: 410.5017
Epoch 322/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.8863 - val_loss: 411.6079
Epoch 323/10000
4/4 [==============================] - 0s 9ms/step - loss: 54.35

...

4/4 [==============================] - 0s 9ms/step - loss: 6.9866 - val_loss: 297.5738
Epoch 1490/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.1375 - val_loss: 303.1339
Epoch 1491/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.0236 - val_loss: 295.2727
Epoch 1492/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.1489 - val_loss: 305.7330
Epoch 1493/10000
4/4 [==============================] - 0s 9ms/step - loss: 35.2103 - val_loss: 304.3310
Epoch 1494/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.4348 - val_loss: 302.3089
Epoch 1495/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.1386 - val_loss: 308.9235
Epoch 1496/10000
4/4 [==============================] - 0s 11ms/step - loss: 12.7935 - val_loss: 306.4921
Epoch 1497/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.1804 - val_loss: 319.2218
Epoch 1498/10000
4/4 [==============================] - 0s 9ms/step - loss:

4/4 [==============================] - 0s 9ms/step - loss: 6.2962 - val_loss: 316.9310
Epoch 1568/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.3281 - val_loss: 320.6713
Epoch 1569/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.6352 - val_loss: 315.1964
Epoch 1570/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.7695 - val_loss: 307.6134
Epoch 1571/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.6988 - val_loss: 315.1253
Epoch 1572/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.3019 - val_loss: 307.3988
Epoch 1573/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.7675 - val_loss: 319.3668
Epoch 1574/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.6259 - val_loss: 305.2773
Epoch 1575/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.0000 - val_loss: 309.4308
Epoch 1576/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.58

4/4 [==============================] - 0s 9ms/step - loss: 11.2754 - val_loss: 344.8150
Epoch 1646/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.0074 - val_loss: 334.1338
Epoch 1647/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.5970 - val_loss: 342.3531
Epoch 1648/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.8783 - val_loss: 348.5018
Epoch 1649/10000
4/4 [==============================] - 0s 9ms/step - loss: 13.4047 - val_loss: 332.7003
Epoch 1650/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.9316 - val_loss: 317.7858
Epoch 1651/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.0357 - val_loss: 330.5455
Epoch 1652/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.8902 - val_loss: 326.0807
Epoch 1653/10000
4/4 [==============================] - 0s 9ms/step - loss: 8.0695 - val_loss: 327.6830
Epoch 1654/10000
4/4 [==============================] - 0s 9ms/step - loss:

4/4 [==============================] - 0s 9ms/step - loss: 7.8807 - val_loss: 289.9374
Epoch 1724/10000
4/4 [==============================] - 0s 9ms/step - loss: 12.4200 - val_loss: 292.6376
Epoch 1725/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.1538 - val_loss: 296.1082
Epoch 1726/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.6515 - val_loss: 296.2469
Epoch 1727/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.5766 - val_loss: 297.6540
Epoch 1728/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.9354 - val_loss: 298.6510
Epoch 1729/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.2922 - val_loss: 296.4614
Epoch 1730/10000
4/4 [==============================] - 0s 9ms/step - loss: 11.5538 - val_loss: 302.9822
Epoch 1731/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.5028 - val_loss: 296.1324
Epoch 1732/10000
4/4 [==============================] - 0s 9ms/step - loss: 

4/4 [==============================] - 0s 9ms/step - loss: 7.9032 - val_loss: 337.1848
Epoch 1802/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.6562 - val_loss: 337.7088
Epoch 1803/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.1055 - val_loss: 343.5181
Epoch 1804/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.0484 - val_loss: 351.6756
Epoch 1805/10000
4/4 [==============================] - 0s 9ms/step - loss: 6.7083 - val_loss: 350.2397
Epoch 1806/10000
4/4 [==============================] - 0s 9ms/step - loss: 9.2024 - val_loss: 353.9202
Epoch 1807/10000
4/4 [==============================] - 0s 9ms/step - loss: 15.3492 - val_loss: 357.5170
Epoch 1808/10000
4/4 [==============================] - 0s 9ms/step - loss: 10.5391 - val_loss: 366.2485
Epoch 1809/10000
4/4 [==============================] - 0s 9ms/step - loss: 17.8057 - val_loss: 377.1802
Epoch 1810/10000
4/4 [==============================] - 0s 9ms/step - loss: 2

In [38]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [39]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,201.097733,202.608002,190.161438,165.102982,162.722549,162.425949,162.19487,163.23909,163.62912,162.617661,169.655518,187.476654,186.678421,178.361099,166.120071,163.138565,163.585159,163.737503,164.532272,164.362625,164.569733,164.947098,167.748764,172.136856,178.066818,171.175568,166.437927,164.25972,164.310669,165.5578
Target,205.551,230.558,209.621,210.984,189.889,143.721,176.275,152.632,177.876,181.384,160.12,236.951,206.557,224.27,213.871,211.066,182.899,159.002,178.918,161.936,173.792,176.627,207.991,185.784,234.176,226.996,218.141,234.939,199.619,168.797
Error,4.453262,27.949997,19.459564,45.881012,27.166458,18.704956,14.080124,10.607086,14.246887,18.766342,9.535522,49.47435,19.878586,45.908905,47.750931,47.927429,19.313843,4.735504,14.385727,2.42662,9.222275,11.679901,40.242233,13.647141,56.109177,55.820435,51.703079,70.679276,35.308334,3.239197


In [40]:
display(mae)
display(mape)

27.01014

0.1305099

In [41]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [42]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'